In [ ]:
!pip3 install benepar
!pip3 install summa
!pip3 install nltk
!pip3 install spacy
!python3 -m spacy download en
!pip3 install scipy
!pip3 install sentence-transformers
!pip3 install transformers
!pip3 install tensorflow
!pip3 install torch

In [16]:
import requests
import json
from summa.summarizer import summarize
import benepar
import string
import nltk
from nltk import tokenize
from nltk.tokenize import sent_tokenize
import re
from random import shuffle
import tensorflow as tf
import transformers
import torch
import summa

In [17]:
!pip3 install cython
!pip3 install benepar

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 2.3.0
    Uninstalling transformers-2.3.0:
      Successfully uninstalled transformers-2.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 0.2.5.1 requires transformers==2.3.0, but you have transformers 4.38.2 which is incompatible.


In [9]:

!benepar.download('benepar_en3')

zsh:1: unknown file attribute: b


In [18]:
import spacy
nlp = spacy.load('en_core_web_sm') 
nltk.download('punkt')
benepar.download()
benepar_parser = benepar.Parser("benepar_en3")
import summa


[nltk_data] Downloading package punkt to /Users/goodwiinz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


showing info https://kitaev.com/benepar/index.xml


In [19]:
file_path = "text.txt"
def read_file(file_path):
  with open(file_path,'r') as content_file:
    content = content_file.read()
    return content
text = read_file(file_path)


In [20]:
from string import punctuation

def preprocess(sentences):
  output = []
  for sent in sentences:
    single_quotes_present = len(re.findall(r"['][\w\s.:;,!?\\-]+[']",sent))>0
    double_quotes_present = len(re.findall(r'["][\w\s.:;,!?\\-]+["]',sent))>0
    question_present = "?" in sent
    if single_quotes_present or double_quotes_present or question_present:
      continue
    else:
      output.append(sent.strip(punctuation))
  return output


def get_candidate_sents(r_text,ratio = .3):
  candidate_sents = summarize(r_text,ratio)
  candidate_sents_list = tokenize.sent_tokenize(candidate_sents)
  candidate_sents_list = [re.split(r'[:;]+',x)[0] for x in candidate_sents_list ]
  filtered_list_short_sentences = [sent for sent in candidate_sents_list if len(sent)>30 and len(sent)<150]
  return filtered_list_short_sentences


cand_sent = get_candidate_sents(text)
filter_quotes_and_questions = preprocess(cand_sent)
for each_sent in filter_quotes_and_questions:
  print(each_sent)
  print('\n')



def get_flattend(t):
  sent_str_final = None
  if t is not None:
    sent_str = [" ".join(x.leaves()) for x in list(t)]
    sent_str_final = [" ".join(sent_str)]
    sent_str_final = sent_str_final[0]
    return sent_str_final

def get_termination_portion(main_string, sub_string):
    combined_sub_string = sub_string.replace(" ","")
    main_string_list = main_string.split()
    last_index = len(main_string_list)
    for i in range(last_index):
        check_string_list = main_string_list[i:]
        check_string = "".join(check_string_list)
        check_string = check_string.replace(" ","")
        if check_string == combined_sub_string:
            return " ".join(main_string_list[:i])
    return None

def get_right_most_VP_or_NP(parse_tree, last_NP = None, last_VP = None):
  if len(parse_tree.leaves()) == 1:
    return get_flattend(last_NP) , get_flattend(last_VP)
  last_subtree = parse_tree[-1]
  if last_subtree.label() == "NP":
    last_NP = last_subtree
  elif last_subtree.label() == "VP":
    last_vp = last_subtree
  return get_right_most_VP_or_NP(last_subtree,last_NP,last_VP)

def get_sentence_completions(key_sentences):
    sentence_completion_dict = {}
    for individual_sentence in filter_quotes_and_questions:
        sentence = individual_sentence.rstrip('?:!.,;')
        tree = benepar_parser.parse(sentence)
        last_nounphrase, last_verbphrase =  get_right_most_VP_or_NP(tree)
        phrases= []
        if last_verbphrase is not None:
            verbphrase_string = get_termination_portion(sentence,last_verbphrase)
            phrases.append(verbphrase_string)
        if last_nounphrase is not None:
            nounphrase_string = get_termination_portion(sentence,last_nounphrase)
            phrases.append(nounphrase_string)

        longest_phrase =  sorted(phrases, key=len,reverse= True)
        if len(longest_phrase) == 2:
            first_sent_len = len(longest_phrase[0].split())
            second_sentence_len = len(longest_phrase[1].split())
            if (first_sent_len - second_sentence_len) > 4:
                del longest_phrase[1]
                
        if len(longest_phrase)>0:
            sentence_completion_dict[sentence]=longest_phrase
    return sentence_completion_dict

If a local government tax is legally dedicated for one or more specific purposes it is a special tax


53] Proposition 218 also requires certain taxes relating to real property (e.g., parcel taxes) be levied as special taxes


Section 2 of Article XIII C contains the voter approval requirements for local government taxes


Under Proposition 218, every local government tax is either a general tax or a special tax


55] Proposition 218 does not allow a local tax to be a hybrid tax


62] The applicable electorate for conducting a local tax election is generally the registered voters of the local government


63][64]
New local government taxes require voter approval under Proposition 218


In [21]:
sent_completion_dict = get_sentence_completions(filter_quotes_and_questions)
print(sent_completion_dict)


from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2",pad_token_id=tokenizer.eos_token_id)
from sentence_transformers import SentenceTransformer
model_BERT = SentenceTransformer('bert-base-nli-mean-tokens')

from nltk import tokenize
import scipy
torch.manual_seed(2020)


def sort_by_similarity(original_sentence,generated_sentences_list):
    # Each sentence is encoded as a 1-D vector with 768 columns
    sentence_embeddings = model_BERT.encode(generated_sentences_list)

    queries = [original_sentence]
    query_embeddings = model_BERT.encode(queries)
    # Find the top sentences of the corpus for each query sentence based on cosine similarity
    number_top_matches = len(generated_sentences_list)

    dissimilar_sentences = []

    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])


        for idx, distance in reversed(results[0:number_top_matches]):
            score = 1-distance
            if score < 0.9:
                dissimilar_sentences.append(generated_sentences_list[idx].strip())
           
    sorted_dissimilar_sentences = sorted(dissimilar_sentences, key=len)
    
    return sorted_dissimilar_sentences[:3]
    

def generate_sentences(partial_sentence,full_sentence):
    input_ids = torch.tensor([tokenizer.encode(partial_sentence)])
    maximum_length = len(partial_sentence.split())+80

    # Actiavte top_k sampling and top_p sampling with only from 90% most likely words
    sample_outputs = model.generate(
        input_ids, 
        do_sample=True, 
        max_length=maximum_length, 
        top_p=0.90, # 0.85 
        top_k=50,   #0.30
        repetition_penalty  = 10.0,
        num_return_sequences=10
    )
    generated_sentences=[]
    for i, sample_output in enumerate(sample_outputs):
        decoded_sentences = tokenizer.decode(sample_output, skip_special_tokens=True)
        decoded_sentences_list = tokenize.sent_tokenize(decoded_sentences)
        generated_sentences.append(decoded_sentences_list[0])
        
    top_3_sentences = sort_by_similarity(full_sentence,generated_sentences)
    
    return top_3_sentences




You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'If a local government tax is legally dedicated for one or more specific purposes it is a special tax': ['If a local government tax is legally dedicated for one or more specific purposes it is'], '53] Proposition 218 also requires certain taxes relating to real property (e.g., parcel taxes) be levied as special taxes': ['53] Proposition 218 also requires certain taxes relating to real property (e.g., parcel taxes) be levied as'], 'Section 2 of Article XIII C contains the voter approval requirements for local government taxes': ['Section 2 of Article XIII C contains the voter approval requirements for'], 'Under Proposition 218, every local government tax is either a general tax or a special tax': ['Under Proposition 218, every local government tax is either a general tax or'], '55] Proposition 218 does not allow a local tax to be a hybrid tax': ['55] Proposition 218 does not allow a local tax to be'], '62] The applicable electorate for conducting a local tax election is generally the r

100%|██████████| 405M/405M [06:52<00:00, 982kB/s]  


In [ ]:
# from collections import defaultdict

# index = 1
# choice_list = ["a)","b)","c)","d)","e)","f)"]
# True_Sentence_out = []

# results = []
# for key_sentence in sent_completion_dict:
    
#     false_sentences = []
#     partial_sentences = sent_completion_dict[key_sentence]
#     print("  **True Sentences (GPT-2 Generated)**")
#     True_Sentence = "%s)"%(str(index))+ str(partial_sentences)
#     temp = {"sentence":partial_sentences[0],"false_sentences":[]}
#     result['true'] = True_Sentence
 
    
    
#     for partial_sent in partial_sentences:
#         false_sents = generate_sentences(partial_sent,key_sentence)
#         false_sentences.extend(false_sents)
        


        

#     temp = {"sentence":partial_sentences[0],"false_sentences":false_sentences}
#     print(temp)
#     results.append(temp)
    
    
#     index = index+1

    

In [22]:
import json

def store(sent_completion_dict):
    index = 1

    results = []
    for key_sentence in sent_completion_dict:
    
        false_sentences = []
        partial_sentences = sent_completion_dict[key_sentence]
        temp = {"sentence":partial_sentences[0],"false_sentences":[]}
    
        for partial_sent in partial_sentences:
            false_sents = generate_sentences(partial_sent,key_sentence)
            false_sentences.extend(false_sents)

        temp = {"sentence":partial_sentences[0],"false_sentences":false_sentences}
        results.append(temp)
    json_object = json.dumps(results, indent = 4)
    return json_object

In [23]:
x = store(sent_completion_dict)
print(x)

[
    {
        "sentence": "If a local government tax is legally dedicated for one or more specific purposes it is",
        "false_sentences": [
            "If a local government tax is legally dedicated for one or more specific purposes it is not taxable to the taxpayer.",
            "If a local government tax is legally dedicated for one or more specific purposes it is not necessary to provide special exemption.",
            "If a local government tax is legally dedicated for one or more specific purposes it is also authorized by statute to levy the same."
        ]
    },
    {
        "sentence": "53] Proposition 218 also requires certain taxes relating to real property (e.g., parcel taxes) be levied as",
        "false_sentences": [
            "53] Proposition 218 also requires certain taxes relating to real property (e.g., parcel taxes) be levied as an abatement of future expenditures by taxpayers who do not have a state-issued permit and are able, in good faith for the pur